In [1]:
import hardy.recognition.cnn as cnn
import hardy.recognition.tuner as tuner
from hardy.handling import pre_processing as preprocessing
from hardy.handling import handling as handling
from hardy.handling import to_catalogue as to_catalogue
from hardy.hardy import classifier_wrapper

Using TensorFlow backend.


In [2]:
import numpy as np

In [3]:
import pickle

In [4]:
import random

In [5]:
image_data = to_catalogue.save_load_data('../classifier_wrapper/rgb_Rex_Imy', file_extension='.npy', load=True)

In [6]:
config_path = '../classifier_wrapper/hardy/recognition/'

In [7]:
test_set_filenames = preprocessing.hold_out_test_set(path='../200504_csv_EIS_simulation/', number_of_files_per_class=300,
                                                     file_extension='.csv', classes=['noise', ''])

In [8]:
classifier_wrapper('../200504_csv_EIS_simulation/', test_set_filenames, '../classifier_wrapper/test_run_9',
                   config_path, image_data=image_data, classifier='tuner',
                   iterator_mode='arrays', split= 0.1, target_size= (80,80),
                   batch_size=32, image_path=None,
                   project_name='tuner_run_9')

INFO:tensorflow:Reloading Oracle from existing project ./tuner_run_9../classifier_wrapper/test_run_9/oracle.json
INFO:tensorflow:Reloading Tuner from ./tuner_run_9../classifier_wrapper/test_run_9/tuner0.json
INFO:tensorflow:Oracle triggered exit
Confusion Matrix 

[[296   4]
 [ 39 260]]

 Classification Report
              precision    recall  f1-score   support

           0       0.88      0.99      0.93       300
           1       0.98      0.87      0.92       299

    accuracy                           0.93       599
   macro avg       0.93      0.93      0.93       599
weighted avg       0.93      0.93      0.93       599

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../200504_csv_EIS_simulation/200605/tuner_run_9/../classifier_wrapper/test_run_9/best_model/assets


In [ ]:
#test_set_filenames = preprocessing.hold_out_test_set(path, number_of_files_per_class=500, classes=[])


def classifier_wrapper(input_path, test_set_filenames, run_name, config_path, image_data=None,classifier='tuner',
                       iterator_mode='arrays', split= 0.1, target_size= (80,80),
                       batch_size=32, image_path=None, classes=['class_1','class_2'],
                       project_name='tuner_run', **kwarg):
    '''
    
    '''
    if iterator_mode == 'arrays':
        
        assert image_data, 'No image_data list provided'
        
        test_set_list, learning_set_list = to_catalogue.data_set_split(image_data, test_set_filenames)
        
        training_set, validation_set = to_catalogue.learning_set(image_list=learning_set_list, split=split, target_size=target_size,
                                                                 iterator_mode='arrays', batch_size=batch_size)
        
        test_set = to_catalogue.test_set(image_list=test_set_list, target_size=target_size,
                                         iterator_mode='arrays', batch_size=batch_size)
    else:
        
        assert image_path, 'no path to the image folders was provided'
        
        training_set, validation_set = to_catalogue.learning_set(image_path, plit=split, target_size=target_size,
                                                                 iterator_mode='from_directory', batch_size=batch_size,
                                                                 classes=classes)
        print(validation_set)
        test_set = to_catalogue.test_set(path, target_size=target_size,  classes=classes,
                                         iterator_mode='from_directory', batch_size=batch_size,)
    
    if classifier == 'tuner':
#         warn search_function, 'no search function provided, using default RandomSearch'
        tuner.build_param(config_path)
        tuner_model= tuner.run_tuner(training_set, validation_set, project_name= project_name + run_name)
        model, history, metrics = tuner.best_model(tuner_model, training_set, validation_set, test_set)
        output_path= preprocessing.save_to_folder(input_path, project_name, run_name)
        conf_matrix, report = cnn.report_on_metrics(model, test_set) 
        tuner.report_generation(model, history, metrics, output_path , tuner=tuner_model, save_model=True)
    else:
        model, history = cnn.build_model(training_set, validation_set, config_path=config_path)
        metrics = cnn.evaluate_model(model, test_set)

        output_path= preprocessing.save_to_folder(input_path, project_name, run_name)
        conf_matrix, report = cnn.report_on_metrics(model, test_set) 
        tuner.report_generation(model, history, metrics, output_path , tuner=None, save_model=True, config_path=config_path)
    
    return

In [1]:
import hardy.run_hardy as run

Using TensorFlow backend.


In [2]:
raw_data_path = '../eisy/examples/simulation_data/'

In [3]:
tform_config_path = './hardy/arbitrage/tform_config.yaml'

In [4]:
classifier_config_path = './hardy/recognition/'

In [5]:
run.hardy_multi_transform(raw_data_path, tform_config_path, 'test_run1', classifier_config_path,
                          iterator_mode='arrays', classes=['noise', ''], project_name='multi_transform')

Successfully Loaded 11 Transforms to Try!
Processing Data...	From 9002 Files:
Found 9000 CSVs...
Found 3 Labels, Only Expected 2...
	3500 Files of Label : one
	4500 Files of Label : noise
	1000 Files of Label : spread
Loaded	9000 of 9002	Files	 at rate of 57 Files per Second
	 Success!	 About 2.63 Minutes...
Making rgb Images from Data...	

C:\Users\meryp\Desktop\DS_Class\project\hardy\hardy\handling\visualization.py:65: RuntimeWarning: invalid value encountered in true_divide
  normalized_image[:, :, i] = img / (np.amax(img, axis=0))


Success in 98.23seconds!
That Took 4.67 Min !


MemoryError: Unable to allocate array with shape (8400, 80, 80, 3) and data type float32

In [12]:
import numpy as np
image_labels = [ 'noisy', 'not_noisy', 'noisy', 'not_noisy' ,'noisy', 'extra']
# image_data = [( 'file_1', [1,2,3,4], 'not_noisy'), ( 'file_2', [1,2,3,4], 'noisy'), ( 'file_3', [1,2,3,4], 'not_noisy'), ( 'file_4', [1,2,3,4], 'noisy')]

In [13]:
np.unique(image_labels)

array(['extra', 'noisy', 'not_noisy'], dtype='<U9')

In [14]:
for i, label in enumerate(np.unique(image_labels)):
    for j in range(len(image_labels)):
        if image_labels[j]== label:
            image_labels[j] = i
        
print(image_labels)

[1, 2, 1, 2, 1, 0]
